# Mining testimonial fragments of the Holocaust

**Experience domain:**

### Load the necessary libraries

In [92]:
import sys; sys.path.insert(0, '..')
import itertools

In [93]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [94]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [95]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [96]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [97]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [98]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [99]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [100]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [101]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [102]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [103]:
main_node = "breathe"
delete_main_node(main_node)
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [104]:
query = '[lemma="breathe"]'

In [105]:
#result = topic_concordancer.main(query,window=25,topicn=25)

### Print the key topics

for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

### Analyze documents

i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

## Testimonial fragments

### 1.  

In [106]:
lemmas = ["breathe","move"]

In [107]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="breathe"][]{0,25}[lemma="move"])|([lemma="move"][]{0,25}[lemma="breathe"])


In [108]:
domain_term = "not to move"

In [109]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [110]:
fragment_1 = {}
fragment_1['original_sentence'] = "And then they shut the trap door, told us not to move, not to breathe, not to say a word, not to sneeze. Our lives depended on it."
fragment_1['label']="(..) told us not to move, not to breathe, not to say a word, not to sneeze. Our lives depended on it."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22then%22%5D+%5B%22they%22%5D+%5B%22shut%22%5D+%5B%22the%22%5D+%5B%22trap%22%5D+%5B%22door%22%5D+%5B%5D+%5B%22told%22%5D+%5B%22us%22%5D+%5B%22not%22%5D+%5B%22to%22%5D+%5B%22move%22%5D+%5B%5D+%5B%22not%22%5D+%5B%22to%22%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22not%22%5D+%5B%22to%22%5D+%5B%22say%22%5D+%5B%22a%22%5D+%5B%22word%22%5D+%5B%5D+%5B%22not%22%5D+%5B%22to%22%5D+%5B%22sneeze%22%5D+%5B%5D+%5B%22Our%22%5D+%5B%22lives%22%5D+%5B%22depended%22%5D+%5B%22on%22%5D+%5B%22it%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And then they shut the trap door , told us not to move , not to breathe , not to say a word , not to sneeze . Our lives depended on it . ', 'right': '', 'complete_match': 'And then they shut the trap door , told us not to move , not to breathe , not to say a word , not to sneeze . Our lives depended on it . ', 'testimony_id': 'usc_shoah_1490

In [111]:
fragment_2 = {}
fragment_2['original_sentence'] = "You didn't dare breathe. You were standing still. You didn't dare move, because the SS woman was coming along, and she was picking out the people."
fragment_2['label']="You didn't dare breathe. You were standing still. You didn't dare move (..)."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22dare%22%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22were%22%5D+%5B%22standing%22%5D+%5B%22still%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22did%22%5D+%5B%5D+%5B%22dare%22%5D+%5B%22move%22%5D+%5B%5D+%5B%22because%22%5D+%5B%22the%22%5D+%5B%22SS%22%5D+%5B%22woman%22%5D+%5B%22was%22%5D+%5B%22coming%22%5D+%5B%22along%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22she%22%5D+%5B%22was%22%5D+%5B%22picking%22%5D+%5B%22out%22%5D+%5B%22the%22%5D+%5B%22people%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "You did n't dare breathe . You were standing still . You did n't dare move , because the SS woman was coming along , and she was picking out the people . ", 'right': '', 'complete_match': "You did n't dare breathe . You were standing still . You did n't dare move , because the SS woman was coming along , and she was picking out the people . ", 'te

In [112]:
fragment_3 = {}
fragment_3['original_sentence'] = "they told me to be very quiet and not to move and I remember feeling that, that I wa suffocating and I couldn't breathe"
fragment_3['label']=".(..)they told me to be very quiet and not to move and I remember feeling that, that I wa suffocating and I couldn't breathe(..)."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22they%22%5D+%5B%22told%22%5D+%5B%22me%22%5D+%5B%22to%22%5D+%5B%22be%22%5D+%5B%22very%22%5D+%5B%22quiet%22%5D+%5B%22and%22%5D+%5B%22not%22%5D+%5B%22to%22%5D+%5B%22move%22%5D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%22remember%22%5D+%5B%22feeling%22%5D+%5B%22that%22%5D+%5B%5D+%5B%22that%22%5D+%5B%22I%22%5D+%5B%22wa%22%5D+%5B%22suffocating%22%5D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22breathe%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "they told me to be very quiet and not to move and I remember feeling that , that I wa suffocating and I could n't breathe ", 'right': '', 'complete_match': "they told me to be very quiet and not to move and I remember feeling that , that I wa suffocating and I could n't breathe ", 'testimony_id': 'irn511022', 'shelfmark': ['USHMM RG-50.471*0005'], 'token_start': 2012, 'token_end': 2038}


In [113]:
fragment_4 = {}
fragment_4['original_sentence'] = "Whispering, don’t move, don’t cough, don’t breathe, nothing."
fragment_4['label']= "Whispering, don’t move, don’t cough, don’t breathe, nothing."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Whispering%22%5D+%5B%5D+%5B%22do%22%5D+%5B%5D+%5B%22move%22%5D+%5B%5D+%5B%22do%22%5D+%5B%5D+%5B%22cough%22%5D+%5B%5D+%5B%22do%22%5D+%5B%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22nothing%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Whispering , do n’t move , do n’t cough , do n’t breathe , nothing . ', 'right': '', 'complete_match': 'Whispering , do n’t move , do n’t cough , do n’t breathe , nothing . ', 'testimony_id': 'irn39788', 'shelfmark': ['USHMM RG-50.030*0541'], 'token_start': 7601, 'token_end': 7617}


In [114]:
add_testimonial_fragments(fragments)

### 2.  

In [115]:
lemmas = ["afraid","breathe"]

In [116]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="afraid"][]{0,10}[lemma="breathe"])|([lemma="breathe"][]{0,10}[lemma="afraid"])


In [117]:
domain_term = "afraid"

In [118]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [119]:
fragment_1 = {}
fragment_1['original_sentence'] = "You couldn't breathe. I was afraid I was to breathe, because to make noise, they would hear you."
fragment_1['label']="You couldn't breathe. I was afraid I was to breathe, because to make noise, they would hear you."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22afraid%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22to%22%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22because%22%5D+%5B%22to%22%5D+%5B%22make%22%5D+%5B%22noise%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22would%22%5D+%5B%22hear%22%5D+%5B%22you%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "You could n't breathe . I was afraid I was to breathe , because to make noise , they would hear you . ", 'right': '', 'complete_match': "You could n't breathe . I was afraid I was to breathe , because to make noise , they would hear you . ", 'testimony_id': 'HVT-72', 'shelfmark': ['Fortunoff HVT-72'], 'token_start': 7282, 'token_end': 7305}


In [120]:
fragment_2 = {}
fragment_2['original_sentence'] = "We were afraid to breathe, they hear us breathe."
fragment_2['label']="We were afraid to breathe, they hear us breathe."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22were%22%5D+%5B%22afraid%22%5D+%5B%22to%22%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22hear%22%5D+%5B%22us%22%5D+%5B%22breathe%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'We were afraid to breathe , they hear us breathe . ', 'right': '', 'complete_match': 'We were afraid to breathe , they hear us breathe . ', 'testimony_id': 'irn44160', 'shelfmark': ['USHMM RG-50.637*0003'], 'token_start': 4659, 'token_end': 4670}


In [121]:
fragment_3 = {}
fragment_3['original_sentence'] = "Oh, yeah, standing with the guns pointing towards you. You were afraid to breathe."
fragment_3['label']="Oh, yeah, standing with the guns pointing towards you. You were afraid to breathe."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Oh%22%5D+%5B%5D+%5B%22yeah%22%5D+%5B%5D+%5B%22standing%22%5D+%5B%22with%22%5D+%5B%22the%22%5D+%5B%22guns%22%5D+%5B%22pointing%22%5D+%5B%22towards%22%5D+%5B%22you%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22were%22%5D+%5B%22afraid%22%5D+%5B%22to%22%5D+%5B%22breathe%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Oh , yeah , standing with the guns pointing towards you . You were afraid to breathe . ', 'right': '', 'complete_match': 'Oh , yeah , standing with the guns pointing towards you . You were afraid to breathe . ', 'testimony_id': 'usc_shoah_1491', 'shelfmark': ['USC 1491'], 'token_start': 5768, 'token_end': 5786}


In [122]:
fragment_4 = {}
fragment_4['original_sentence'] = "And I was under that bed, lying flat, afraid to breathe even."
fragment_4['label']= "And I was under that bed, lying flat, afraid to breathe even."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22under%22%5D+%5B%22that%22%5D+%5B%22bed%22%5D+%5B%5D+%5B%22lying%22%5D+%5B%22flat%22%5D+%5B%5D+%5B%22afraid%22%5D+%5B%22to%22%5D+%5B%22breathe%22%5D+%5B%22even%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And I was under that bed , lying flat , afraid to breathe even . ', 'right': '', 'complete_match': 'And I was under that bed , lying flat , afraid to breathe even . ', 'testimony_id': 'usc_shoah_9590', 'shelfmark': ['USC 9590'], 'token_start': 8807, 'token_end': 8822}


In [123]:
fragment_5 = {}
fragment_5['original_sentence'] = "Because of fear, we were absolutely silent, almost afraid to breathe (..)."
fragment_5['label']= "Because of fear, we were absolutely silent, almost afraid to breathe, "
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Because%22%5D+%5B%22of%22%5D+%5B%22fear%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22absolutely%22%5D+%5B%22silent%22%5D+%5B%5D+%5B%22almost%22%5D+%5B%22afraid%22%5D+%5B%22to%22%5D+%5B%22breathe%22%5D+%5B%5D%7B0%2C3%7D+%5B%5D+%5B%5D+%5B%5D%7B0%2C3%7D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Because of fear , we were absolutely silent , almost afraid to breathe , because he ', 'right': '', 'complete_match': 'Because of fear , we were absolutely silent , almost afraid to breathe , because he ', 'testimony_id': 'usc_shoah_8784', 'shelfmark': ['USC 8784'], 'token_start': 7959, 'token_end': 7975}


In [124]:
add_testimonial_fragments(fragments)

### 3.  

In [125]:
lemmas = ["choke","breathe"]

In [126]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="choke"][]{0,10}[lemma="breathe"])|([lemma="breathe"][]{0,10}[lemma="choke"])


In [127]:
domain_term = "choke"

In [128]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [129]:
fragment_1 = {}
fragment_1['original_sentence'] = "And she couldn't take it out, and the baby couldn't breathe, and the baby choked."
fragment_1['label']="And she couldn't take it out, and the baby couldn't breathe, and the baby choked."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22she%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22take%22%5D+%5B%22it%22%5D+%5B%22out%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22the%22%5D+%5B%22baby%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22the%22%5D+%5B%22baby%22%5D+%5B%22choked%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And she could n't take it out , and the baby could n't breathe , and the baby choked . ", 'right': '', 'complete_match': "And she could n't take it out , and the baby could n't breathe , and the baby choked . ", 'testimony_id': 'usc_shoah_25381', 'shelfmark': ['USC 25381'], 'token_start': 15683, 'token_end': 15703}


In [130]:
fragment_2 = {}
fragment_2['original_sentence'] = "And she choked the baby. Got choked. Couldn't breathe because they were a long time in the forest."
fragment_2['label']="And she choked the baby. Got choked. Couldn't breathe because they were a long time in the forest."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22she%22%5D+%5B%22choked%22%5D+%5B%22the%22%5D+%5B%22baby%22%5D+%5B%5D+%5B%22Got%22%5D+%5B%22choked%22%5D+%5B%5D+%5B%22Could%22%5D+%5B%5D+%5B%22breathe%22%5D+%5B%22because%22%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22a%22%5D+%5B%22long%22%5D+%5B%22time%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22forest%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And she choked the baby . Got choked . Could n't breathe because they were a long time in the forest . ", 'right': '', 'complete_match': "And she choked the baby . Got choked . Could n't breathe because they were a long time in the forest . ", 'testimony_id': 'usc_shoah_25381', 'shelfmark': ['USC 25381'], 'token_start': 15652, 'token_end': 15674}


In [131]:
fragment_3 = {}
fragment_3['original_sentence'] = "You could almost choke, breathing that air there."
fragment_3['label']="You could almost choke, breathing that air there."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22could%22%5D+%5B%22almost%22%5D+%5B%22choke%22%5D+%5B%5D+%5B%22breathing%22%5D+%5B%22that%22%5D+%5B%22air%22%5D+%5B%22there%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You could almost choke , breathing that air there . ', 'right': '', 'complete_match': 'You could almost choke , breathing that air there . ', 'testimony_id': 'irn504798', 'shelfmark': ['USHMM RG-50.030*0303'], 'token_start': 17107, 'token_end': 17117}


In [132]:
fragment_4 = {}
fragment_4['original_sentence'] = "I had always suffered from anxieties and sometimes they were so severe I had the feeling I was choking in the middle of the night and couldn't breathe"
fragment_4['label']= "I had always suffered from anxieties (..) I had the feeling I was choking in the middle of the night and couldn't breathe "
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22had%22%5D+%5B%22always%22%5D+%5B%22suffered%22%5D+%5B%22from%22%5D+%5B%22anxieties%22%5D+%5B%22and%22%5D+%5B%22sometimes%22%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22so%22%5D+%5B%22severe%22%5D+%5B%22I%22%5D+%5B%22had%22%5D+%5B%22the%22%5D+%5B%22feeling%22%5D+%5B%22I%22%5D+%5B%22was%22%5D+%5B%22choking%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22middle%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22night%22%5D+%5B%22and%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22breathe%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "I had always suffered from anxieties and sometimes they were so severe I had the feeling I was choking in the middle of the night and could n't breathe ", 'right': '', 'complete_match': "I had always suffered from anxieties and sometimes they were so severe I had the feeling I was choking in the middle of the night and could n't breathe ", 'testimony_id

In [133]:
add_testimonial_fragments(fragments)

### 4.  

In [134]:
lemmas = []

In [135]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

()


In [136]:
domain_term = "smell"

In [137]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [138]:
fragment_1 = {}
fragment_1['original_sentence'] = "I remember we couldn't breathe. The smell was terrible"
fragment_1['label']="I remember we couldn't breathe. The smell was terrible (..)"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22I%22%5D+%5B%22remember%22%5D+%5B%22we%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22The%22%5D+%5B%22smell%22%5D+%5B%22was%22%5D+%5B%22terrible%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "I remember we could n't breathe . The smell was terrible ", 'right': '', 'complete_match': "I remember we could n't breathe . The smell was terrible ", 'testimony_id': 'irn505573', 'shelfmark': ['USHMM RG-50.042*0020'], 'token_start': 2741, 'token_end': 2752}


In [139]:
fragment_2 = {}
fragment_2['original_sentence'] = "It was a terrible smell. Months and months we have to smell...we always have to breathe it in"
fragment_2['label']="It was a terrible smell. Months and months we have to smell...we always have to breathe it in (..)."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22It%22%5D+%5B%22was%22%5D+%5B%22a%22%5D+%5B%22terrible%22%5D+%5B%22smell%22%5D+%5B%5D+%5B%22Months%22%5D+%5B%22and%22%5D+%5B%22months%22%5D+%5B%22we%22%5D+%5B%22have%22%5D+%5B%22to%22%5D+%5B%22smell%22%5D+%5B%5D%7B0%2C50%7D+%5B%22we%22%5D+%5B%22always%22%5D+%5B%22have%22%5D+%5B%22to%22%5D+%5B%22breathe%22%5D+%5B%22it%22%5D+%5B%22in%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'It was a terrible smell . Months and months we have to smell ... we always have to breathe it in ', 'right': '', 'complete_match': 'It was a terrible smell . Months and months we have to smell ... we always have to breathe it in ', 'testimony_id': 'irn508637', 'shelfmark': ['USHMM RG-50.462*0016'], 'token_start': 27734, 'token_end': 27755}


In [140]:
fragment_3 = {}
fragment_3['original_sentence'] = "were eating and breathing this-- this-- this smell"
fragment_3['label']="we were eating and breathing this smell"
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22were%22%5D+%5B%22eating%22%5D+%5B%22and%22%5D+%5B%22breathing%22%5D+%5B%22this%22%5D+%5B%5D%7B0%2C3%7D+%5B%22this%22%5D+%5B%5D%7B0%2C3%7D+%5B%22this%22%5D+%5B%22smell%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'were eating and breathing this -- this -- this smell ', 'right': '', 'complete_match': 'were eating and breathing this -- this -- this smell ', 'testimony_id': 'usc_shoah_7506', 'shelfmark': ['USC 7506'], 'token_start': 20797, 'token_end': 20807}


In [141]:
fragment_4 = {}
fragment_4['original_sentence'] = "My nostrils fill with the dank smell of death, but I have to breathe."
fragment_4['label']= "My nostrils fill with the dank smell of death, but I have to breathe."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22My%22%5D+%5B%22nostrils%22%5D+%5B%22fill%22%5D+%5B%22with%22%5D+%5B%22the%22%5D+%5B%22dank%22%5D+%5B%22smell%22%5D+%5B%22of%22%5D+%5B%22death%22%5D+%5B%5D+%5B%22but%22%5D+%5B%22I%22%5D+%5B%22have%22%5D+%5B%22to%22%5D+%5B%22breathe%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'My nostrils fill with the dank smell of death , but I have to breathe . ', 'right': '', 'complete_match': 'My nostrils fill with the dank smell of death , but I have to breathe . ', 'testimony_id': 'usc_shoah_322', 'shelfmark': ['USC 322'], 'token_start': 883, 'token_end': 899}


In [142]:
fragment_5 = {}
fragment_5['original_sentence'] = " the stench, the smell of the burning flesh. I just could not breathe."
fragment_5['label']= "(..)the stench, the smell of the burning flesh. I just could not breathe."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22the%22%5D+%5B%22stench%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22smell%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22burning%22%5D+%5B%22flesh%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22just%22%5D+%5B%22could%22%5D+%5B%22not%22%5D+%5B%22breathe%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'the stench , the smell of the burning flesh . I just could not breathe . ', 'right': '', 'complete_match': 'the stench , the smell of the burning flesh . I just could not breathe . ', 'testimony_id': 'usc_shoah_23730', 'shelfmark': ['USC 23730'], 'token_start': 9964, 'token_end': 9980}


In [143]:
add_testimonial_fragments(fragments)

### 5.  

In [144]:
lemmas = ["breathe","die"]

In [145]:
query = create_contextual_query(lemmas,context_length=25)
print (query)

([lemma="breathe"][]{0,25}[lemma="die"])|([lemma="die"][]{0,25}[lemma="breathe"])


In [146]:
domain_term = "die"

In [147]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [148]:
fragment_1 = {}
fragment_1['original_sentence'] = " The people couldn't breathe. There's a lot of people died right then and there."
fragment_1['label']=" The people couldn't breathe. There's a lot of people died right then and there. "
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22The%22%5D+%5B%22people%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22There%22%5D+%5B%5D+%5B%22a%22%5D+%5B%22lot%22%5D+%5B%22of%22%5D+%5B%22people%22%5D+%5B%22died%22%5D+%5B%22right%22%5D+%5B%22then%22%5D+%5B%22and%22%5D+%5B%22there%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "The people could n't breathe . There 's a lot of people died right then and there . ", 'right': '', 'complete_match': "The people could n't breathe . There 's a lot of people died right then and there . ", 'testimony_id': 'usc_shoah_1775', 'shelfmark': ['USC 1775'], 'token_start': 4075, 'token_end': 4093}


In [149]:
fragment_2 = {}
fragment_2['original_sentence'] = "the girls were dying right in front of us. And-- and it was so heavy to breathe, you know, in that closed-in cattle car."
fragment_2['label']="(..) the girls were dying right in front of us. And-- and it was so heavy to breathe, you know, in that closed-in cattle car. "
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22the%22%5D+%5B%22girls%22%5D+%5B%22were%22%5D+%5B%22dying%22%5D+%5B%22right%22%5D+%5B%22in%22%5D+%5B%22front%22%5D+%5B%22of%22%5D+%5B%22us%22%5D+%5B%5D+%5B%22And%22%5D+%5B%5D%7B0%2C3%7D+%5B%22and%22%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22so%22%5D+%5B%22heavy%22%5D+%5B%22to%22%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D+%5B%22in%22%5D+%5B%22that%22%5D+%5B%5D%7B0%2C3%7D+%5B%22cattle%22%5D+%5B%22car%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'the girls were dying right in front of us . And -- and it was so heavy to breathe , you know , in that closed-in cattle car . ', 'right': '', 'complete_match': 'the girls were dying right in front of us . And -- and it was so heavy to breathe , you know , in that closed-in cattle car . ', 'testimony_id': 'usc_shoah_3949', 'shelfmark': ['USC 3949'], 'token_start': 14579, 'token_end': 14608}


In [150]:
fragment_3 = {}
fragment_3['original_sentence'] = "my father couldn't breathe. He was dying right then and there."
fragment_3['label']="(..) my father couldn't breathe. He was dying right then and there."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22my%22%5D+%5B%22father%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22He%22%5D+%5B%22was%22%5D+%5B%22dying%22%5D+%5B%22right%22%5D+%5B%22then%22%5D+%5B%22and%22%5D+%5B%22there%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "my father could n't breathe . He was dying right then and there . ", 'right': '', 'complete_match': "my father could n't breathe . He was dying right then and there . ", 'testimony_id': 'usc_shoah_19111', 'shelfmark': ['USC 19111'], 'token_start': 17261, 'token_end': 17275}


In [151]:
fragment_4 = {}
fragment_4['original_sentence'] = "You could hardly breathe. You were face to face. We had people that finally passed away, died."
fragment_4['label']= "You could hardly breathe. You were face to face. We had people that finally passed away, died."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22You%22%5D+%5B%22could%22%5D+%5B%22hardly%22%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22You%22%5D+%5B%22were%22%5D+%5B%22face%22%5D+%5B%22to%22%5D+%5B%22face%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22had%22%5D+%5B%22people%22%5D+%5B%22that%22%5D+%5B%22finally%22%5D+%5B%22passed%22%5D+%5B%22away%22%5D+%5B%5D+%5B%22died%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'You could hardly breathe . You were face to face . We had people that finally passed away , died . ', 'right': '', 'complete_match': 'You could hardly breathe . You were face to face . We had people that finally passed away , died . ', 'testimony_id': 'usc_shoah_4284', 'shelfmark': ['USC 4284'], 'token_start': 16226, 'token_end': 16247}


In [152]:
fragment_5 = {}
fragment_5['original_sentence'] = "We couldn't even breathe. Quite a few older people die."
fragment_5['label']= "We couldn't even breathe. Quite a few older people die."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22even%22%5D+%5B%22breathe%22%5D+%5B%5D+%5B%22Quite%22%5D+%5B%22a%22%5D+%5B%22few%22%5D+%5B%22older%22%5D+%5B%22people%22%5D+%5B%22die%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "We could n't even breathe . Quite a few older people die . ", 'right': '', 'complete_match': "We could n't even breathe . Quite a few older people die . ", 'testimony_id': 'usc_shoah_7160', 'shelfmark': ['USC 7160'], 'token_start': 1159, 'token_end': 1172}


In [153]:
add_testimonial_fragments(fragments)